# Installation
CNGI documentation is located here:
[https://cngi-prototype.readthedocs.io/en/latest/index.html](https://cngi-prototype.readthedocs.io/en/latest/index.html)

Google Colab requires specific older versions of some packages such as Pandas and Dask, so we will install CNGI without its normal dependencies and then manually install each dependency afterwards.

Normally, you would want to leave out the --no-dependencies option

In [1]:
import os

print("installing cngi (takes a few minutes)...")
os.system("apt-get install libgfortran3")
os.system("pip install --extra-index-url https://casa-pip.nrao.edu/repository/pypi-group/simple casatools")
os.system("pip install cngi-prototype==0.0.9 --no-dependencies")
os.system("pip install --upgrade dask")
os.system("pip install --upgrade xarray")
os.system("pip install --upgrade zarr")

print("downloading MeasurementSet from CASAguide First Look at Imaging...")
os.system("wget https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar")
os.system("tar -xvf sis14_twhya_calibrated_flagged.ms.tar")

print('complete')

installing cngi (takes a few minutes)...
downloading MeasurementSet from CASAguide First Look at Imaging...
complete


# Initialize the Processing Environment
This is a bit limited with Colab, and the bokeh dashboard doesn't work

In [2]:
from cngi.direct import InitializeFramework
client = InitializeFramework(2,'6GB',False)
client

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/125/1,Cluster Workers: 2 Cores: 2 Memory: 12.00 GB


# Convert an MS to xarray zarr
Takes some time


In [5]:
from cngi.conversion import ms_to_zarr

ms_to_zarr('sis14_twhya_calibrated_flagged.ms')

processing ddi 0: chunks=1, size=53717
completed ddi 0
Complete.


# Open an xarray zarr based MS

(todo) Retrieve a summary of the xarray zarr MS file. 

Then create a new xarray Dataset from it.

This Dataset is the common data structure passed around to most other CNGI functions.

In [7]:
from cngi.ms import summarizeFile
from cngi.dio import read_zarr

# returns summary as a pandas dataframe
#mssummary = summarizeFile('sis14_twhya_calibrated_flagged.pq')
#print(mssummary[['ddi','row_count_estimate','col_count','size_GB']])

# there is only one ddi in the MS, but pretend there are more and one is chosen
ddi = 0 #mssummary.ddi.values[0]

# here we create the dask dataframe for use in other CNGI functions
xds = read_zarr('sis14_twhya_calibrated_flagged.zarr',ddi=ddi)

# examine the start of the dataframe 
# note that the column selection should be made in to a convenience function
#cols = [col for col in ddf.columns.values if col not in list(ddf.columns.values[ddf.columns.str.match('(FLAG\d)|(R|IDATA\d)')])]
#ddf[cols].head()
xds

<xarray.Dataset>
Dimensions:         (chans: 384, pols: 2, rows: 80563, uvw: 3)
Coordinates:
    ANTENNA1        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    ANTENNA2        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    ARRAY_ID        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    EXPOSURE        (rows) float64 dask.array<chunksize=(53717,), meta=np.ndarray>
    FEED1           (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    FEED2           (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    FIELD_ID        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    FLAG_ROW        (rows) int64 dask.array<chunksize=(53717,), meta=np.ndarray>
    INTERVAL        (rows) float64 dask.array<chunksize=(53717,), meta=np.ndarray>
    OBSERVATION_ID  (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    PROCESSOR_ID    (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    SCAN_NUM